In [2]:
import bs4
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

In [3]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [4]:
# 1 또는 2 중 선택하여 실행

# 1
# option_list = []
# for i in range(len_option) :
#     option = str(soup_option.find_all('li')[i]).split('\">')[1].split('<')[0]
#     option_list.append(option)
# print(option_list)

# 2
import pickle
with open('./files/option_list.pkl', 'rb') as f :
    option_list = pickle.load(f)
print(option_list)

['썬루프', '제논라이트(HID)', '알루미늄휠', '전동접이식미러', '주간주행등(DLR)', '파노라마썬루프', '루프랙', 'LED리어램프', '하이빔 어시스트', '고스트 도어 클로징', '자동슬라이딩도어', '전동사이드스탭', '어댑티드헤드램프', '열선시트', '통풍시트', '파워핸들', '핸들 리모컨', '가죽시트', '운전석전동시트', '조수석전동시트', '뒷좌석전동시트', '메모리시트', '안마시트', '슈퍼비전계기판', '패들시프트', '하이패스', '메모리시트(동승석)', '열선시트(뒷좌석)', '엠비언트라이트', '워크인시트', '전동햇빛가리개', '통풍시트(동승석)', '통풍시트(뒷좌석)', '후방감지센서', '사이드&amp;커튼에어백', '운전석에어백', '조수석에어백', '후방카메라', '어라운드뷰', '블랙박스', 'ABS', 'ECS', 'TCS', '차체자세제어장치', '차선이탈경보장치', '도난경보기', '타이어 공기압감지', '무릎에어백', '주차감지센서(전방)', '전방카메라', '자동긴급제동(AEB)', '스마트키', '파워윈도우', '자동도어잠금', '풀오토에어컨', '오토라이트', '자동주차시스템', '전자식파킹브레이크', 'HUD', 'ECM룸밀러', '크루즈컨트롤', '전동트렁크', '핸들열선', '무선도어 잠금장치', '레인센서와이퍼', '스탑앤고', '무선충전', '내비게이션', 'CD플레이어', 'CD체인저', 'AV시스템', 'MP3', 'AUX', 'USB', '핸즈프리', 'iPod 단자', '뒷좌석모니터', '스마트폰미러링', '내비게이션 (비순정)', '블루투스']


In [5]:
# import pickle
# with open('option_list.pkl', 'wb') as f :
#     pickle.dump(option_list, f)

In [6]:
# 파일 경로 정의
file_paths = ["morning01.pkl", "morning02.pkl", "morning03.pkl", "morning04.pkl", "morning05.pkl", "morning06.pkl"]

# 파일 불러오기 및 변수에 할당
for i, file_path in enumerate(file_paths, start=1):
    with open('./files/morning/' + file_path, 'rb') as f:
        data = pickle.load(f)
        exec(f"m0{i} = data")

In [7]:
len(m06)

500

In [8]:
columns = ['name', 'year', 'fuel', 'distance', 'color']
columns.extend(option_list)
columns.append('price')
len(columns)

86

In [15]:
# df_01 = pd.DataFrame(columns=columns)
# df_02 = pd.DataFrame(columns=columns)
# df_03 = pd.DataFrame(columns=columns)
# df_04 = pd.DataFrame(columns=columns)
df_05 = pd.DataFrame(columns=columns)
# df_06 = pd.DataFrame(columns=columns)

In [16]:
(pd.Series(m05).value_counts() >= 2).sum()

0

In [17]:
m05[0]

'25578230'

In [18]:
for car_seq in m05 :
    url = 'https://www.kbchachacha.com/public/car/detail.kbc?carSeq='
    url = url + car_seq
    iframe = 'layerPop_layerCarOptionView'
    try :
        driver.get(url)
        time.sleep(1)
        button = driver.find_element(By.XPATH, '//*[@id="btnCarOptionMore"]')
        button.click()
        time.sleep(1)

        tmp = [0] * len(df_05.columns)
        option_on_list = []

        html_main = driver.page_source
        soup_main = bs4.BeautifulSoup(html_main, 'html.parser')
    
        driver.switch_to.frame(iframe)
        html_option = driver.page_source
        soup_option = bs4.BeautifulSoup(html_option, 'html.parser')
        len_option = len(soup_option.find_all('li'))


        src1 = soup_main.find('strong', class_='car-buy-name')
        name1 = str(src1).split(')')[1].split('<')[0]
        name2 = str(src1).split(')')[1].split('<')[1].split('>')[1]
        name = name1 + ' ' + name2


        info_table = soup_main.find('table', class_='detail-info-table')
        year = int(str(info_table).split('년')[0][-2:])
        distance = int(str(info_table).split('km')[0].split('td>')[-1].replace(',', ''))
        fuel = str(info_table).split('연료</th>\n<td>')[1].split('</td>')[0]
        color = str(info_table).split('색상</th>\n<td>')[1].split('</td>')[0]


        src2 = soup_main.select_one('div > dl > dd > strong')
        price = int(str(src2.text[:-2]).replace(',', ''))

        option_len = len(soup_option.find_all('li', class_='on'))

        tmp[0] = name
        tmp[1] = year
        tmp[2] = fuel
        tmp[3] = distance
        tmp[4] = color
        tmp[-1] = price

        for i in range(option_len) :
            name = str(soup_option.find_all('li', class_='on')[i]).split('\">')[1].split('<')[0]
            option_on_list.append(name)

        for option in option_on_list :
            idx = option_list.index(option)
            tmp[idx + 5] = 1   

        df_05.loc[m05.index(car_seq)] = tmp
        time.sleep(3)
    except :
        continue

In [19]:
df_05

,name,year,fuel,distance,color,썬루프,제논라이트(HID),알루미늄휠,전동접이식미러,주간주행등(DLR),...,MP3,AUX,USB,핸즈프리,iPod 단자,뒷좌석모니터,스마트폰미러링,내비게이션 (비순정),블루투스,price
0,기아 all new 모닝 디럭스 바이퓨얼,12,LPG,171949,회색,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,300
1,기아 all new 모닝 디럭스 스페셜,11,가솔린,113275,베이지색,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,420
2,기아 뉴모닝 LX 기본형,8,가솔린,133408,회색,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,230
5,기아 all new 모닝 럭셔리,12,가솔린,100564,검정,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,430
6,기아 all new 모닝 럭셔리,12,가솔린,115848,진주색,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,기아 뉴모닝 LX 스페셜,10,가솔린,100000,쥐색,0,0,1,0,0,...,0,1,1,0,0,0,0,0,0,250
496,기아 all new 모닝 디럭스 트렌디,13,가솔린,91417,흰색,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,470
497,기아 all new 모닝 디럭스 트렌디,12,가솔린,121550,회색,0,0,1,1,0,...,1,1,1,0,0,0,0,1,0,999
498,기아 뉴모닝 L,8,가솔린,162449,파랑색,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,150


In [20]:
df_05.to_csv('./files/morning/csv/m05.csv', index=False)